Todo:

2. Merge train and valid splits into 1 and perform 80-20 split
    1. Essentialy you want to take 5400 images from each class from valid and move it to test
3. Prepare the whole pipeline
    1. Data augmentation
        - First take the unaugmented original dataset and proceed
        -  Augment the data with a predefined seed for each of the following techniques: rotation, flipping, contrast, brightness change, random erasing
    2. Choose hyperparameters
        - For the hyper-parameters related to the training process we chose batch size, learning rate, and number of epochs
        - For the hyper-parameters related to Regularization we decided to use L2 Regularization (Weight Decay) and Dropout Rate
    3. Train each of the prepared models on the augmented dataset for a chosen augmentation technique
    4. Test and collect data regarding models’ performance on the augmented dataset
    5. Repeat several times (>=3) from 5.
    6. Choose different values for hyperparameters and start from 5.
    7. Choose the next augmentation technique and start from 4.
    8. Repeat the process starting from 3. several times (>=3) with a different seed each time

## Move 5400 images from each class from valid to train

In [19]:
import os
rootdir = './Dataset/valid'

for subdir, dirs, files in os.walk(rootdir):
    if subdir != rootdir:
        for subsubdir, subdirs, files in os.walk(subdir):
            if len(files) < 5400:
                break;
            for i in range(5400):
                os.rename(os.path.join(os.path.join("./Dataset/valid",os.path.basename(subsubdir)),files[i]), os.path.join(os.path.join("./Dataset/train",os.path.basename(subsubdir)),files[i]))